# CNN (Convolutional Neural Network)

## Core Architecture
1. Convolutional Layer
* Function: Extracts features via filters, producing feature maps.
* Operation: Element-wise multiplication between filter weights and input regions, followed by activation.
2. Pooling Layer
* Function: Down samples feature maps to reduce dimensionality and prevent overfitting.
* Methods: Max pooling (selects maximum value) or average pooling (computes regional average).
3. Fully connected Layer
* Function: Maps high-level features to output labels (e.g., classification).
* Position: Typically at the network's end.

In [1]:
import torch
import torchdata
print(torch.__version__)
print(torchdata.__version__)

2.0.1
0.5.1


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 超参数设置
batch_size = 64
num_epochs = 5
learning_rate = 0.001

# 数据预处理（以CIFAR-10为例）
transform = transforms.Compose([
    transforms.ToTensor(),  # 转为Tensor [0,1]
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 归一化到[-1,1]
])

# 加载数据集
train_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False)

# 定义CNN模型
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        
        # 卷积层1: 输入3通道，输出16通道，卷积核3x3
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        # ReLU激活
        self.relu = nn.ReLU()
        # 最大池化层1: 2x2窗口，步长2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # 卷积层2: 输入16通道，输出32通道，卷积核3x3
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
        # 全连接层1: 输入32 * 8 * 8（计算见下方forward），输出512
        self.fc1 = nn.Linear(32 * 8 * 8, 512)
        # 全连接层2: 输出10类（CIFAR-10）
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        # 输入x维度: [batch_size, 3, 32, 32]
        
        # 卷积层1 → ReLU → 池化
        x = self.conv1(x)  # 输出维度: [batch_size, 16, 32, 32]
        x = self.relu(x)
        x = self.pool(x)   # 输出维度: [batch_size, 16, 16, 16]
        
        # 卷积层2 → ReLU → 池化
        x = self.conv2(x)  # 输出维度: [batch_size, 32, 16, 16]
        x = self.relu(x)
        x = self.pool(x)   # 输出维度: [batch_size, 32, 8, 8]
        
        # 展平操作: [batch_size, 32 * 8 * 8]
        x = x.view(-1, 32 * 8 * 8)
        
        # 全连接层1 → ReLU
        x = self.fc1(x)    # 输出维度: [batch_size, 512]
        x = self.relu(x)
        
        # 全连接层2（无激活函数，损失函数包含Softmax）
        x = self.fc2(x)    # 输出维度: [batch_size, 10]
        return x

# 初始化模型、损失函数、优化器
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 训练循环
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播与优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

# 测试模型
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/5], Step [100/782], Loss: 1.6158
Epoch [1/5], Step [200/782], Loss: 1.2451
Epoch [1/5], Step [300/782], Loss: 1.4432
Epoch [1/5], Step [400/782], Loss: 1.4878
Epoch [1/5], Step [500/782], Loss: 1.4096
Epoch [1/5], Step [600/782], Loss: 1.2399
Epoch [1/5], Step [700/782], Loss: 1.0525
Epoch [2/5], Step [100/782], Loss: 1.1116
Epoch [2/5], Step [200/782], Loss: 1.1358
Epoch [2/5], Step [300/782], Loss: 1.1427
Epoch [2/5], Step [400/782], Loss: 1.0619
Epoch [2/5], Step [500/782], Loss: 0.9216
Epoch [2/5], Step [600/782], Loss: 1.0173
Epoch [2/5], Step [700/782], Loss: 0.8095
Epoch [3/5], Step [100/782], Loss: 0.8400
Epoch [3/5], Step [200/782], Loss: 0.8372
Epoch [3/5], Step [300/782], Loss: 0.8566
Epoch [3/5], Step [400/782], Loss: 0.7306
Epoch [3/5], Step [500/782], Loss: 0.9003
Epoch [3/5], Step [600/782], Loss: 0.8303
Epoch [3/5], Step [700/782], Loss: 0.7791
Epoch [4/5], Step [100/782], Loss: 0.7870
